In [99]:
import jax

In [100]:
def open_data():
    nietzsche_txt = open('/Users/mikaumana/PycharmProjects/NietzscheGPT/new_nietzsche.txt', 'r', encoding='utf-8')
    nietzsche = nietzsche_txt.read()
    return nietzsche

nietzsche = open_data()

In [101]:
def sort_characters(data):
    chararacters = sorted(list(set(data)))
    return chararacters

all_characters = sort_characters(nietzsche)
vocab_size = len(all_characters)

In [102]:
def encode(text, all_characters):
    encoded_text = []
    for c in text:
        num = all_characters.index(c)
        encoded_text.append(num)
    return encoded_text

In [103]:
def decode(encoded_text, all_characters):
    text = []
    for n in encoded_text:
        char = all_characters[n]
        text.append(char)
    string = ''.join([str(item) for item in text])
    return string

In [104]:
def encoding_nietzsche(data, all_characters):
    encoded_data = jax.numpy.asarray(encode(data, all_characters))
    print(encoded_data.shape, encoded_data.dtype)
    return encoded_data

data = encoding_nietzsche(nietzsche, all_characters)

(3396780,) int32


In [105]:
def splitting_data(data, split):
    n = int(split * len(data))
    training_data = data[:n]
    validation_data = data[n:]
    return training_data, validation_data

training_data, validation_data = splitting_data(data, 0.9)
print(validation_data.__class__)

<class 'jaxlib.xla_extension.ArrayImpl'>


In [106]:
import jax.random
batchsize = 4
blocksize = 8
training_data[:blocksize + 1]
key = jax.random.PRNGKey(42)

def get_batch(command, batchsize, key, blocksize):
    train_batches_data = []
    eval_batches_data = []
    if command == 'train':
        b_data = training_data
    else:
        b_data = validation_data
    for x in range(batchsize):
        key, subkey = jax.random.split(key)
        pos = jax.random.randint(key = subkey, shape = (), minval = 0, maxval = (len(b_data) - blocksize))
        batch_data = b_data[pos:pos + blocksize]
        train_batches_data.append(batch_data)
        batch_data = b_data[pos+1: pos + blocksize +1]
        eval_batches_data.append(batch_data)
        key = subkey
    
    train_batches_data = numpy.stack(train_batches_data)
    eval_batches_data = numpy.stack(eval_batches_data)
    
    return train_batches_data, eval_batches_data
    
train, evals = get_batch('train', batchsize, key, blocksize)
print(train)
        
    

[[71 70  1 71 62  1 79 65]
 [57 76  1 76 64 61  1 79]
 [ 1 76 71  1 76 64 61  0]
 [57 74 81  1 59 71 74 71]]


In [116]:
from flax import linen as nn
import optax

class Bigramm(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(self.vocab_size, self.vocab_size)
    
    def __call__(self, train, evals= 'None'):
        
        logits = self.token_embedding_table(train)
        if evals == 'None':
            mean_loss = None
        else:
            b, t, c = logits.shape
            logits = logits.reshape((b*t, c))
            labels = evals.reshape((b*t))
            labels_one_hot = jax.nn.one_hot(labels, num_classes=self.vocab_size)
            loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
            mean_loss = loss.mean()
        return logits, mean_loss
    
    def generate(self, train, length):
        key = jax.random.PRNGKey(43)
        for x in length:
            logits, loss = self(train)
            logits = logits[:,-1,:]
            probabilities = jax.nn.softmax(logits)
            key, subkey = jax.random.split(key)
            next_logit = jax.random.choice(subkey, logits, p = probabilities)
            key = subkey
            train = jax.numpy.concatenate((train, next_logit), axis = 1)
        return train
            
    
m = Bigramm(vocab_size = vocab_size)
params = m.init(jax.random.PRNGKey(0), train, evals) 
s, loss = m.apply(params, train, evals)
print(decode(m.generate(train = jax.numpy.zeros((1,1)), length = 100)), all_characters)
print(s.shape)
print(loss)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [109]:
print(decode(data, all_characters))


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

